In [1]:
from datasets import load_dataset

# Hugging Face Hubのllm-book/jawiki-paragraphsのリポジトリから
# Wikipediaの段落テキストのデータを読み込む
paragraph_dataset = load_dataset(
    "llm-book/jawiki-paragraphs", split="train"
)
     

Generating train split:   0%|          | 0/9668476 [00:00<?, ? examples/s]

In [3]:
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 9668476
})


In [4]:
from pprint import pprint

pprint(paragraph_dataset[0])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}


In [15]:
import random 
random.random()

0.7358104407127645

In [17]:
import random 

paragraph_dataset = paragraph_dataset.filter(
    lambda example: example["paragraph_index"] == 0 and random.random() < 0.01

)

Filter:   0%|          | 0/1339236 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModel, AutoTokenizer

# Hugging Face Hubにアップロードされた
# 教師なしSimCSEのトークナイザとエンコーダを読み込む
model_name = "llm-book/bert-base-japanese-v3-unsup-simcse-jawiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [18]:
paragraph_dataset

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 13307
})

In [8]:

# 読み込んだモデルをGPUのメモリに移動させる
device = "cpu"
encoder = encoder.to(device)

In [9]:
import numpy as np
import torch
import torch.nn.functional as F

def embed_texts(texts: list[str]) -> np.ndarray:
    """SimCSEのモデルを用いてテキストの埋め込みを計算"""
    # テキストにトークナイザを適用
    tokenized_texts = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)

    # トークナイズされたテキストをベクトルに変換
    with torch.inference_mode():
        with torch.cuda.amp.autocast():
            encoded_texts = encoder(
                **tokenized_texts
            ).last_hidden_state[:, 0]

    # ベクトルをNumPyのarrayに変換
    emb = encoded_texts.cpu().numpy().astype(np.float32)
    # ベクトルのノルムが1になるように正規化
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
    return emb

In [19]:

# 段落データのすべての事例に埋め込みを付与する
paragraph_dataset = paragraph_dataset.map(
    lambda examples: {
        "embeddings": list(embed_texts(examples["text"]))
    },
    batched=True,
)

Map:   0%|          | 0/13307 [00:00<?, ? examples/s]

/Users/satsuki/github/tech-notes/llm-book/src/.venv/lib/python3.11/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


KeyboardInterrupt: 

In [20]:
import faiss

# ベクトルの次元数をエンコーダの設定値から取り出す
emb_dim = encoder.config.hidden_size
# ベクトルの次元数を指定して空のFaissインデックスを作成する
index = faiss.IndexFlatIP(emb_dim)
# 段落データの"embeddings"フィールドのベクトルからFaissインデックスを構築する
paragraph_dataset.add_faiss_index("embeddings", custom_index=index)
     

ValueError: Columns ['embeddings'] not in the dataset. Current columns in the dataset: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag']

In [ ]:

query_text = "日本語は、主に日本で話されている言語である。"

# 最近傍探索を実行し、類似度上位10件の事例とスコアを取得する
scores, retrieved_examples = paragraph_dataset.get_nearest_examples(
    "embeddings", embed_texts([query_text])[0], k=10
)
# 取得した事例の内容をスコアとともに表示する
titles = retrieved_examples["title"]
texts = retrieved_examples["text"]
for score, title, text in zip(scores, titles, texts):
    print(score, title, text)
     